In [95]:
#Import dependencies
import quandl
import pandas as pd
import numpy as np 
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split

In [96]:
#Retrieve data from Quandl
quandl.ApiConfig.api_key = "1sNXfWjF5c7B4CLEZZyT"

In [97]:
#Assign start and end dates form the stock
start = pd.to_datetime('2014-10-17')
end = pd.to_datetime('2019-10-17')
data_TGT = quandl.get('EOD/AMZN', start_date='2014-10-17', end_date='2019-10-17')
data_TGT.tail()

,Open,High,Low,Close,Volume,Dividend,Split,Adj_Open,Adj_High,Adj_Low,Adj_Close,Adj_Volume
Date,,,,,,,,,,,,
2019-10-11,1742.92,1745.45,1729.86,1731.92,3279534.0,0.0,1.0,1742.92,1745.45,1729.86,1731.92,3279534.0
2019-10-14,1728.91,1741.89,1722.00,1736.43,1928898.0,0.0,1.0,1728.91,1741.89,1722.00,1736.43,1928898.0
2019-10-15,1742.14,1776.45,1740.62,1767.38,3129244.0,0.0,1.0,1742.14,1776.45,1740.62,1767.38,3129244.0
2019-10-16,1773.33,1786.24,1770.52,1777.43,2804068.0,0.0,1.0,1773.33,1786.24,1770.52,1777.43,2804068.0
2019-10-17,1796.49,1798.85,1782.02,1787.48,2713773.0,0.0,1.0,1796.49,1798.85,1782.02,1787.48,2713773.0


In [98]:
# Get the Adjusted Close Price 
data_TGT = data_TGT[['Adj_Close']] 
# Take a look at the new data 
print(data_TGT.head())

            Adj_Close
Date                 
2014-10-17     303.64
2014-10-20     306.21
2014-10-21     315.33
2014-10-22     312.97
2014-10-23     313.18


In [99]:
# A variable for predicting 'n' days out into the future
forecast_out=30     
#Create another column (the target ) shifted 'n' units up
data_TGT['Prediction'] = data_TGT[['Adj_Close']].shift(-forecast_out)
#print the new data set
print(data_TGT.tail())


            Adj_Close  Prediction
Date                             
2019-10-11    1731.92         NaN
2019-10-14    1736.43         NaN
2019-10-15    1767.38         NaN
2019-10-16    1777.43         NaN
2019-10-17    1787.48         NaN


In [100]:
### Create the independent data set (X)  #######
# Convert the dataframe to a numpy array
X = np.array(data_TGT.drop(['Prediction'],1))

#Remove the last '30' rows
X = X[:-forecast_out]
print(X)

[[ 303.64]
 [ 306.21]
 [ 315.33]
 ...
 [1789.84]
 [1800.62]
 [1840.72]]


In [101]:
### Create the dependent data set (y)  #####
# Convert the dataframe to a numpy array 
y = np.array(data_TGT['Prediction'])
# Get all of the y values except the last '30' rows
y = y[:-forecast_out]
print(y)

[ 326.    326.31  316.5  ... 1767.38 1777.43 1787.48]


In [102]:
# Split the data into 80% training and 20% testing
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [103]:
# Create and train the Support Vector Machine (Regressor) 
svr_rbf = SVR(kernel='rbf', C=1e3, gamma=0.1) 
svr_rbf.fit(x_train, y_train)

SVR(C=1000.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=0.1,
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [104]:
# Testing Model: Score returns the coefficient of determination R^2 of the prediction. 
# The best possible score is 1.0
svm_confidence = svr_rbf.score(x_test, y_test)
print("svm confidence: ", svm_confidence)

svm confidence:  0.9304890027054755


In [105]:
# Create and train the Linear Regression  Model
lr = LinearRegression()
# Train the model
lr.fit(x_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [106]:
# Testing Model: Score returns the coefficient of determination R^2 of the prediction. 
# The best possible score is 1.0
lr_confidence = lr.score(x_test, y_test)
print("lr confidence: ", lr_confidence)

lr confidence:  0.967052027814775


In [107]:
# Set x_forecast equal to the last 30 rows of the original data set from Adj. Close column
x_forecast = np.array(data_TGT.drop(['Prediction'],1))[-forecast_out:]
print(x_forecast)

[[1833.51]
 [1831.35]
 [1820.55]
 [1822.99]
 [1843.55]
 [1839.34]
 [1807.84]
 [1822.55]
 [1817.46]
 [1821.5 ]
 [1794.16]
 [1785.3 ]
 [1741.61]
 [1768.33]
 [1739.84]
 [1725.45]
 [1735.91]
 [1735.65]
 [1713.23]
 [1724.42]
 [1739.65]
 [1732.66]
 [1705.51]
 [1721.99]
 [1720.26]
 [1731.92]
 [1736.43]
 [1767.38]
 [1777.43]
 [1787.48]]


In [108]:
# Print linear regression model predictions for the next '30' days
lr_prediction = lr.predict(x_forecast)
print(lr_prediction)

[1854.77193567 1852.6535574  1842.06166602 1844.45464889 1864.61847173
 1860.48959555 1829.59657904 1844.02312739 1839.03120821 1842.99336017
 1816.18018329 1807.49090944 1764.6427859  1790.84790975 1762.90689259
 1748.79417806 1759.05262101 1758.79763103 1736.80964912 1747.78402546
 1762.72055376 1755.86524629 1729.23840824 1745.4008499  1743.70418582
 1755.13950558 1759.56260096 1789.9162156  1799.77255897 1809.62890233]


In [109]:
# Print support vector regressor model predictions for the next '30' days
svm_prediction = svr_rbf.predict(x_forecast)
print(svm_prediction)

[1843.67548579 1629.67292361 1909.69483584 1856.22481909 1843.62972197
 1815.54125729 1784.89759214 1885.20567077 1942.10438219 1904.0944785
 1806.93625789 1745.24583912 1861.55680608 1754.61131712 1838.44649924
 1813.00964219 1892.43607523 1873.61989966 1885.71913765 1832.92816248
 1845.31297348 1650.70382872 1746.59188211 1598.83065201 1461.93467342
 1668.04024969 1921.88853579 1834.88424207 1832.17183994 1695.38053513]
